In [1]:
import pandas as pd
df = pd.read_csv("combined_data.csv")

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing  import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 170)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtype == "object"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and dataframe[col].dtype != "object"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and dataframe[col].dtype == "object"]
    cat_cols = list(set(cat_cols + num_but_cat) - set(cat_but_car))
    
  
    num_cols = [col for col in dataframe.columns if dataframe[col].dtype in ["int64", "float64"]]
    
    return cat_cols, num_cols, cat_but_car
cat_cols, num_cols, cat_but_car = grab_col_names(df)

print("Kategorik Değişkenler:", cat_cols)
print("Sayısal Değişkenler:", num_cols)
print("Kardinalitesi Yüksek Kategorik Değişkenler:", cat_but_car)

In [ ]:
import polars as pl
df = pl.read_csv("combined_data.csv")
postpaid_missing_auto_payment = df.filter(
    (pl.col("service_type") == "Postpaid") & (pl.col("auto_payment").is_null())
)
print(f"Postpaid olup auto_payment eksik olan satır sayısı: {postpaid_missing_auto_payment.shape[0]}")
print(postpaid_missing_auto_payment.head(10))

In [ ]:
import pandas as pd
df = pd.read_csv("combined_data.csv")
df_copy = df.copy()
df_copy.loc[(df_copy["service_type"] == "Postpaid") & (df_copy["auto_payment"].isnull()) & (df_copy["overdue_payments"] == 0), "auto_payment"] = "True"
df_copy.loc[(df_copy["service_type"] == "Postpaid") & (df_copy["auto_payment"].isnull()) & (df_copy["overdue_payments"] > 0), "auto_payment"] = "False"i
df_copy["tenure"].fillna(df_copy["tenure"].median(), inplace=True)
df_copy["data_usage"].fillna(0, inplace=True)
df_copy["avg_call_duration"].fillna(df_copy["avg_call_duration"].median(), inplace=True)
df_copy["roaming_usage"].fillna(0, inplace=True)
df_copy["call_drops"].fillna(0, inplace=True)
df_copy["monthly_charge"] = df_copy.groupby(["service_type", "auto_payment"])["monthly_charge"].transform(lambda x: x.fillna(x.median()))

df_copy.to_csv("combined_data_filled.csv", index=False)


print(df_copy.isnull().sum())



In [ ]:
df_copy.loc[(df_copy["service_type"] == "Postpaid") & (df_copy["auto_payment"].isnull()) & (df_copy["overdue_payments"] == 0), "auto_payment"] = "True"
df_copy.loc[(df_copy["service_type"] == "Postpaid") & (df_copy["auto_payment"].isnull()) & (df_copy["overdue_payments"] > 0), "auto_payment"] = "False"
df_copy["auto_payment"].fillna("Unknown", inplace=True)



grouped_median = df_copy.groupby(["service_type", "auto_payment"])["monthly_charge"].median().reset_index()
grouped_median.rename(columns={"monthly_charge": "median_monthly_charge"}, inplace=True)


df_copy = df_copy.merge(grouped_median, on=["service_type", "auto_payment"], how="left")


df_copy["monthly_charge"].fillna(df_copy["median_monthly_charge"], inplace=True)

df_copy.drop(columns=["median_monthly_charge"], inplace=True)


df_copy["monthly_charge"].fillna(df_copy["monthly_charge"].median(), inplace=True)


print("Güncellenmiş veri setindeki null değerler:")
print(df_copy.isnull().sum())

df_copy.to_csv("combined_data_filled.csv", index=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder


cat_cols, num_cols, cat_but_car = grab_col_names(df)


def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

binary_cols = [col for col in df.columns if df[col].dtype in ["int64", "object"] and df[col].nunique() == 2]

for col in binary_cols:
    df = label_encoder(df, col)


cat_cols = [col for col in cat_cols if col not in binary_cols and col not in ["churn", "auto_payment"]]

# Sonucu Kontrol Et
print("Binary Değişkenler:", binary_cols)
print("Güncellenmiş Kategorik Değişkenler:", cat_cols)


In [ ]:

unknown_analysis = df_copy[df_copy["auto_payment"] == "Unknown"].groupby(["service_type", "overdue_payments"]).size().reset_index(name="count")
print("'Unknown' değerlerinin dağılımı:")
print(unknown_analysis)

In [ ]:
mport pandas as pd


df_copy = pd.read_csv("combined_data_filled.csv")

for service_type in df_copy["service_type"].unique():
    for overdue in df_copy["overdue_payments"].unique():
  
        mask = (df_copy["service_type"] == service_type) & (df_copy["overdue_payments"] == overdue) & (df_copy["auto_payment"] == "Unknown")
        
   
        auto_payments_in_group = df_copy.loc[(df_copy["service_type"] == service_type) & (df_copy["overdue_payments"] == overdue), "auto_payment"]
        if not auto_payments_in_group.empty:
            mode_value = auto_payments_in_group.mode()
            if not mode_value.empty:
                mode_value = mode_value[0]
                if mode_value == "Unknown":
                    mode_value = False
                print(f"service_type: {service_type}, overdue: {overdue}, mode_value: {mode_value}")
                df_copy.loc[mask, "auto_payment"] = mode_value

df_copy.to_csv("combined_data_filled.csv", index=False)
remaining_unknown = (df_copy["auto_payment"] == "Unknown").sum()
print(f"auto_payment sütununda kalan 'Unknown' değer sayısı: {remaining_unknown}")


In [ ]:
from sklearn.preprocessing import LabelEncoder
cat_cols, num_cols, cat_but_car = grab_col_names(df)
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe


binary_cols = [col for col in df.columns if df[col].dtype in ["int64", "object"] and df[col].nunique() == 2]


for col in binary_cols:
    df = label_encoder(df, col)
cat_cols = [col for col in cat_cols if col not in binary_cols and col not in ["churn", "auto_payment"]]
print("Binary Değişkenler:", binary_cols)
print("Güncellenmiş Kategorik Değişkenler:", cat_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoderme
cat_cols, num_cols, cat_but_car = grab_col_names(df)
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

binary_cols = [col for col in df.columns if df[col].dtype in ["int64", "object"] and df[col].nunique() == 2]


for col in binary_cols:
    df = label_encoder(df, col)


cat_cols = [col for col in cat_cols if col not in binary_cols and col not in ["churn", "auto_payment"]]


print("Binary Değişkenler:", binary_cols)
print("Güncellenmiş Kategorik Değişkenler:", cat_cols)

In [ ]:
import polars as pl
from sklearn.preprocessing import LabelEncoder


df_copy = pl.read_csv("combined_data_filled.csv")


def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe = dataframe.with_column(pl.Series(binary_col, labelencoder.fit_transform(dataframe[binary_col].to_pandas())))
    return dataframe


binary_cols = [col for col in df_copy.columns if df_copy[col].dtype in [pl.Int64, pl.Utf8] and df_copy[col].unique().len() == 2]

for col in binary_cols:
    df_copy = label_encoder(df_copy, col)


df_copy = df_copy.to_pandas()
df_copy = pd.get_dummies(df_copy, columns=["service_type", "apps"], drop_first=True)
df_copy = pl.DataFrame(df_copy)
df_copy.write_csv("combined_data_filled.csv")

print("One-Hot Encoding işlemi tamamlandı ve veriler kalıcı hale getirildi.")


In [ ]:
import polars as pl
df_copy = pl.read_csv("combined_data_filled.csv")
df_copy = df_copy.slice(1, df_copy.shape[0] - 1)
df_copy.write_csv("combined_data_filled.csv")

print("Gereksiz ilk satır silindi ve veriler kaydedildi.")


In [ ]:
import pandas as pd
import polars as pl
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import re

df_copy = pl.read_csv("combined_data_filled.csv")
df_copy = df_copy.slice(1, df_copy.shape[0] - 1)
def clean_column_names(column_names):
    cleaned_names = []
    for col in column_names:
        cleaned_col = re.sub(r'[^A-Za-z0-9_]+', '_', col)
        cleaned_names.append(cleaned_col)
    return cleaned_names

df_copy.columns = clean_column_names(df_copy.columns)
y = df_copy["churn"].to_pandas()
X = df_copy.drop(["churn", "id"]).to_pandas()
stratified_sample_size = sample_size // 2

X_churn_1 = X[y == 1]
y_churn_1 = y[y == 1]
X_churn_0 = X[y == 0]
y_churn_0 = y[y == 0]

X_churn_1_sample, _, y_churn_1_sample, _ = train_test_split(X_churn_1, y_churn_1, train_size=stratified_sample_size, random_state=12345)
X_churn_0_sample, _, y_churn_0_sample, _ = train_test_split(X_churn_0, y_churn_0, train_size=stratified_sample_size, random_state=12345)


X_sample = pd.concat([X_churn_1_sample, X_churn_0_sample], axis=0)
y_sample = pd.concat([y_churn_1_sample, y_churn_0_sample], axis=0)
X_sample.columns = clean_column_names(X_sample.columns)

print(f"Seçilen örneklem boyutu: {X_sample.shape}")
print(f"Churn sınıflarının dağılımı:\n{y_sample.value_counts()}")

models = [
    ('LR', LogisticRegression(random_state=12345)),
    ('CART', DecisionTreeClassifier(random_state=12345)),
    ('RF', RandomForestClassifier(random_state=12345)),
    ('SVM', SVC(gamma='scale', random_state=12345)),
    ('XGB', XGBClassifier(random_state=12345)),
    ('LightGBM', LGBMClassifier(random_state=12345))
]

for name, model in models:
    try:
        cv_results = cross_validate(
            model, X_sample, y_sample, cv=10, 
            scoring=["accuracy", "f1", "roc_auc", "precision", "recall"], 
            error_score='raise'
        )
        print(f"######## {name} ######")
        print(f"Accuracy: {round(cv_results['test_accuracy'].mean(), 4)}")
        print(f"AUC: {round(cv_results['test_roc_auc'].mean(), 4)}")
        print(f"Recall: {round(cv_results['test_recall'].mean(), 4)}")
        print(f"F1: {round(cv_results['test_f1'].mean(), 4)}")
        print(f"Precision: {round(cv_results['test_precision'].mean(), 4)}\n")
    except Exception as e:
        print(f"{name} modelinde hata oluştu: {e}\n")


In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


In [ ]:
def check_outlier (dataframe, col_name): 
    low_limit, up_limit = outlier_thresholds(dataframe, col_name) 
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None): 
        return True 
    else: 
        return False

In [ ]:
def replace_with_thresholds(dataframe, variable, q1=0.05, q3=0.95):
    low_limit, up_limit = outlier_thresholds(dataframe, variable, q1, q3)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [ ]:
for col in num_cols:
    print(col, check_outlier(df, col))
    if check_outlier(df, col):
        replace_with_thresholds(df, col)


In [ ]:

import pandas as pd


df_copy = pl.read_csv("combined_data_filled.csv")


df = df_copy.to_pandas()


df['call_center_calls_per_month'] = df['customer_support_calls'] / (df['tenure'] + 1)


df_copy = pl.DataFrame(df)


df_copy.write_csv("combined_data_filled.csv")

print("Yeni sütun oluşturuldu ve veriler kalıcı olarak kaydedildi.")



In [ ]:
import polars as pl
import pandas as pd
df_copy = pl.read_csv("combined_data_filled.csv")
df = df_copy.to_pandas()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['normalized_satisfaction_score'] = scaler.fit_transform(df[['satisfaction_score']])


df['normalized_support_calls'] = scaler.fit_transform(df[['customer_support_calls']])

df['customer_support_effectiveness_score'] = df['normalized_satisfaction_score'] * df['normalized_support_calls']

In [ ]:

apps_columns = [col for col in df.columns if col.startswith("apps_")]
df['total_apps_used'] = df[apps_columns].sum(axis=1)


In [ ]:

df['AgeGroup'] = pd.cut(df['age'], bins=[0, 25, 50, 75, 100], labels=['Genç', 'Orta Yaşlı', 'Yaşlı', 'Çok Yaşlı'])
df['months_until_next_contract'] = 12 - (df['tenure'] % 12)


In [ ]:

df_copy = pl.DataFrame(df)
df_copy.write_csv("combined_data_filled.csv")

print("Tüm özellikler oluşturuldu ve veriler kalıcı olarak kaydedildi.")

In [ ]:
from sklearn.preprocessing import LabelEncoder
df_copy = pl.read_csv("combined_data_filled.csv")
df = df_copy.to_pandas()
label_encoder = LabelEncoder()
df['AgeGroupEncoded'] = label_encoder.fit_transform(df['AgeGroup'])

df_copy = pl.DataFrame(df)
df_copy.write_csv("combined_data_filled.csv")

print("Yaş grubu sütunu Label Encoding ile dönüştürüldü ve veriler kalıcı olarak kaydedildi.")

In [ ]:
import pandas as pd
import polars as pl
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
df_copy = pl.read_csv("combined_data_filled.csv")

df = df_copy.to_pandas()
y = df["churn"]
X = df.drop(["churn", "id"], axis=1)
X_sample, _, y_sample, _ = train_test_split(X, y, train_size=5000, stratify=y, random_state=17)
rf_model = RandomForestClassifier(random_state=17)
rf_params = {
    "max_depth": [5, 8, None],
    "max_features": [3, 5, 7, "sqrt", "log2"],
    "min_samples_split": [2, 5, 8],
    "n_estimators": [100, 200, 500]
}


rf_best_grid = GridSearchCV(rf_model, rf_params, cv=5, n_jobs=-1, verbose=True).fit(X_sample, y_sample)
print("En iyi parametreler:", rf_best_grid.best_params_)
print("En iyi skor:", rf_best_grid.best_score_)

rf_final = RandomForestClassifier(**rf_best_grid.best_params_, random_state=17).fit(X, y)


cv_results = cross_validate(rf_final, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"])
print("Test Accuracy:", cv_results['test_accuracy'].mean())
print("Test F1 Score:", cv_results['test_f1'].mean())
print("Test ROC AUC Score:", cv_results['test_roc_auc'].mean())


In [ ]:
import pandas as pd
import polars as pl
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate


df_copy = pl.read_csv("combined_data_filled.csv")

r
df = df_copy.to_pandas()

y = df["churn"]
X = df.drop(["churn", "id"], axis=1)

X_sample, _, y_sample, _ = train_test_split(X, y, train_size=5000, stratify=y, random_state=17)
xgboost_model = XGBClassifier(random_state=17)
xgboost_params = {
    "learning_rate": [0.1, 0.01, 0.001],
    "max_depth": [5, 8, 12, 15, 20],
    "n_estimators": [100, 500, 1000],
    "colsample_bytree": [0.5, 0.7, 1]
}

xgboost_best_grid = GridSearchCV(xgboost_model, xgboost_params, cv=5, n_jobs=-1, verbose=True).fit(X_sample, y_sample)
print("En iyi parametreler:", xgboost_best_grid.best_params_)
print("En iyi skor:", xgboost_best_grid.best_score_)

xgboost_final = XGBClassifier(**xgboost_best_grid.best_params_, random_state=17).fit(X_sample, y_sample)


cv_results = cross_validate(xgboost_final, X_sample, y_sample, cv=10, scoring=["accuracy", "f1", "roc_auc"])
print("Test Accuracy:", cv_results['test_accuracy'].mean())
print("Test F1 Score:", cv_results['test_f1'].mean())
print("Test ROC AUC Score:", cv_results['test_roc_auc'].mean())

In [ ]:
import pandas as pd
import polars as pl
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate

df_copy = pl.read_csv("combined_data_filled.csv")

df = df_copy.to_pandas()

df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)


y = df["churn"]
X = df.drop(["churn", "id"], axis=1)

X_sample, _, y_sample, _ = train_test_split(X, y, train_size=1000, stratify=y, random_state=17)
lgbm_model = LGBMClassifier(random_state=17)
lgbm_params = {
    "learning_rate": [0.01, 0.1, 0.001],
    "n_estimators": [100, 300, 500, 1000],
    "colsample_bytree": [0.5, 0.7, 1]
}
lgbm_best_grid = GridSearchCV(lgbm_model, lgbm_params, cv=5, n_jobs=-1, verbose=True).fit(X_sample, y_sample)
print("En iyi parametreler:", lgbm_best_grid.best_params_)
print("En iyi skor:", lgbm_best_grid.best_score_)

lgbm_final = LGBMClassifier(**lgbm_best_grid.best_params_, random_state=17).fit(X_sample, y_sample)

cv_results = cross_validate(lgbm_final, X_sample, y_sample, cv=10, scoring=["accuracy", "f1", "roc_auc"])
print("Test Accuracy:", cv_results['test_accuracy'].mean())
print("Test F1 Score:", cv_results['test_f1'].mean())
print("Test ROC AUC Score:", cv_results['test_roc_auc'].mean())